In [ ]:
import sys
import pandas as pd
import numpy as np
from glob import glob
import os
from types import SimpleNamespace

REPO_DIR = os.path.abspath("..")
sys.path.append(REPO_DIR)

import utils.data_utils as data_utils
import utils.PATHS as PATHS

In [ ]:
LAB_FIELDS = SimpleNamespace(**data_utils.LAB_FIELDS)
DIAG_FIELDS = SimpleNamespace(**data_utils.DIAG_FIELDS)

In [ ]:
# Load data
labs_df = pd.read_csv(os.path.join(PATHS.OUTPUT_DIR, "labs_2015-2022.pkl"))
diag_df = pd.read_csv(os.path.join(PATHS.OUTPUT_DIR, "diag.csv"))

In [ ]:
# range of lab tests that contain key words, test missingness 

# Lab test exploration

In [ ]:
glucose_mask = labs_df[FIELD["TEST_NAME"]].str.contains("glucose", case=False, na=False) & labs_df[FIELD["TEST_NAME"]].str.contains("fasting", case=False, na=False)

In [ ]:
ldl_mask = labs_df[FIELD["TEST_NAME"]].str.contains("ldl", case=False, na=False) # 

In [ ]:
fasting_glucose = labs_df[glucose_mask]

In [ ]:
fasting_glucose[FIELD["TEST_NAME"]].unique()

In [ ]:
ldl = labs_df[ldl_mask]

In [ ]:
ldl[FIELD["TEST_NAME"]].unique()

# Lab test filtering

In [ ]:
glucose_mask = labs_df[FIELD["TEST_NAME"]].str.contains("glucose", case=False, na=False) & labs_df[FIELD["TEST_NAME"]].str.contains("fasting", case=False, na=False)
other_tests_mask = labs_df[FIELD["TEST_NAME"]].str.contains('HBA1C|TRIGLYCERIDES|LDL-CHOLESTEROL,CALCULATED|LDL-CHOLESTEROL,DIRECT', case=False, na=False)

In [ ]:
filtered_labs_df = labs_df
filtered_labs_df['Lab Resulted Order Test Description'] = labs_df['Lab Resulted Order Test Description'].str.replace('GLUCOSE,PLASMA FASTING', 'GLUCOSE FASTING', case=False, regex=True)
filtered_labs_df = filtered_labs_df[filtered_labs_df['Lab Resulted Order Test Description'].str.contains('HBA1C|TRIGLYCERIDES|LDL-CHOLESTEROL,CALCULATED|LDL-CHOLESTEROL,DIRECT|GLUCOSE FASTING', case=False, na=False)]

In [ ]:
##converting all LDL labels to LAD-CHOLESTEROL
filtered_labs_df['Lab Resulted Order Test Description'] = filtered_labs_df['Lab Resulted Order Test Description'].str.replace('LDL-CHOLESTEROL,CALCULATED|LDL-CHOLESTEROL,DIRECT', 'LDL-CHOLESTEROL', case=False,regex=True)
filtered_labs_df

In [ ]:
filtered_labs_df.to_csv(os.path.join(OUTPUT_DIR, "filtered_labs.csv"), index=False)

In [ ]:
filtered_labs_df['Lab Resulted Order Test Description'].unique

# Finalising HbA1C and Triglyceride data

In [ ]:
filtered_labs_df[FIELD["DATE"]] = pd.to_datetime(filtered_labs_df[FIELD["DATE"]])

In [ ]:
filtered_labs_df = filtered_labs_df[~filtered_labs_df['Lab Resulted Order Test Description'].str.contains('TRIGLYCERIDES,FLUID | TRIGLYCERIDES,URINE', case=False, na=False)]

In [ ]:
filtered_labs_df['Lab Resulted Order Test Description'].unique()

# Convert HbA1C IFCC values into percentage values

In [ ]:
filtered_labs_df['Result Value']=pd.to_numeric(filtered_labs_df['Result Value'], errors='coerce')
for index, value in filtered_labs_df['Lab Resulted Order Test Description'].items():
    if value == 'HBA1C IFCC':
        filtered_labs_df.at[index, 'Result Value'] = 0.09148* filtered_labs_df.at[index, 'Result Value']+ 2.152
        filtered_labs_df.at[index, 'Lab Resulted Order Test Description'] = 'HBA1C'
        
filtered_labs_df

In [ ]:
dff = filtered_labs_df[filtered_labs_df['Lab Resulted Order Test Description']=='HBA1c, blood (dual reporting)']
dff